In [ ]:
# Copyright 2023 Google LLC
#
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
#     https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

# Overview

This notebook explores the dataset we will use and does the needed data preprocessing.

**Input:**

[Kaggle Flipkart Product Catalog Dataset](https://www.kaggle.com/datasets/PromptCloudHQ/flipkart-products)

This is a pre-crawled dataset of 20K products, taken as subset of a bigger dataset (more than 5.8 million products) that was created by extracting data from Flipkart.com, a leading Indian eCommerce store.

**Output:**

BigQuery table in a schema compatible for use in downstream notebooks.

# Setup

### Install Dependencies (If Needed)

The list `packages` contains tuples of package import names and install names. If the import name is not found then the install name is used to install quitely for the current user.

In [ ]:
# tuples of (import name, install name)
packages = [
    ('google.cloud.aiplatform', 'google-cloud-aiplatform'),
    ('opendatasets','opendatasets'),
    ('jsonpickle','jsonpickle'),
]

import importlib
install = False
for package in packages:
    if not importlib.util.find_spec(package[0]):
        print(f'installing package {package[1]}')
        install = True
        !pip install {package[1]} -U -q --user

### Restart Kernel (If Installs Occured)

After a kernel restart the code submission can start with the next cell after this one.

In [ ]:
if install:
    import IPython
    app = IPython.Application.instance()
    app.kernel.do_shutdown(True)

### Authenticate

If you are using Colab, you will need to authenticate yourself first. The next cell will check if you are currently using Colab, and will start the authentication process.

In [2]:
import sys
if 'google.colab' in sys.modules:
    from google.colab import auth as google_auth
    google_auth.authenticate_user()

# Dataset

[This](https://www.kaggle.com/datasets/PromptCloudHQ/flipkart-products) is a pre-crawled dataset, taken as subset of a bigger dataset (more than 5.8 million products) that was created by extracting data from [Flipkart](https://www.flipkart.com/), a leading Indian eCommerce store.

Since this data is hosted on Kaggle you will need a Kaggle account to download it.

In [ ]:
import opendatasets as od
od.download("https://www.kaggle.com/datasets/PromptCloudHQ/flipkart-products/download")

In [1]:
import pandas as pd
full_ds = pd.read_csv('flipkart-products/flipkart_com-ecommerce_sample.csv')
full_ds.head()

,uniq_id,crawl_timestamp,product_url,product_name,product_category_tree,pid,retail_price,discounted_price,image,is_FK_Advantage_product,description,product_rating,overall_rating,brand,product_specifications
0,c2d766ca982eca8304150849735ffef9,2016-03-25 22:59:23 +0000,http://www.flipkart.com/alisha-solid-women-s-c...,Alisha Solid Women's Cycling Shorts,"[""Clothing >> Women's Clothing >> Lingerie, Sl...",SRTEH2FF9KEDEFGF,999.0,379.0,"[""http://img5a.flixcart.com/image/short/u/4/a/...",False,Key Features of Alisha Solid Women's Cycling S...,No rating available,No rating available,Alisha,"{""product_specification""=>[{""key""=>""Number of ..."
1,7f7036a6d550aaa89d34c77bd39a5e48,2016-03-25 22:59:23 +0000,http://www.flipkart.com/fabhomedecor-fabric-do...,FabHomeDecor Fabric Double Sofa Bed,"[""Furniture >> Living Room Furniture >> Sofa B...",SBEEH3QGU7MFYJFY,32157.0,22646.0,"[""http://img6a.flixcart.com/image/sofa-bed/j/f...",False,FabHomeDecor Fabric Double Sofa Bed (Finish Co...,No rating available,No rating available,FabHomeDecor,"{""product_specification""=>[{""key""=>""Installati..."
2,f449ec65dcbc041b6ae5e6a32717d01b,2016-03-25 22:59:23 +0000,http://www.flipkart.com/aw-bellies/p/itmeh4grg...,AW Bellies,"[""Footwear >> Women's Footwear >> Ballerinas >...",SHOEH4GRSUBJGZXE,999.0,499.0,"[""http://img5a.flixcart.com/image/shoe/7/z/z/r...",False,Key Features of AW Bellies Sandals Wedges Heel...,No rating available,No rating available,AW,"{""product_specification""=>[{""key""=>""Ideal For""..."
3,0973b37acd0c664e3de26e97e5571454,2016-03-25 22:59:23 +0000,http://www.flipkart.com/alisha-solid-women-s-c...,Alisha Solid Women's Cycling Shorts,"[""Clothing >> Women's Clothing >> Lingerie, Sl...",SRTEH2F6HUZMQ6SJ,699.0,267.0,"[""http://img5a.flixcart.com/image/short/6/2/h/...",False,Key Features of Alisha Solid Women's Cycling S...,No rating available,No rating available,Alisha,"{""product_specification""=>[{""key""=>""Number of ..."
4,bc940ea42ee6bef5ac7cea3fb5cfbee7,2016-03-25 22:59:23 +0000,http://www.flipkart.com/sicons-all-purpose-arn...,Sicons All Purpose Arnica Dog Shampoo,"[""Pet Supplies >> Grooming >> Skin & Coat Care...",PSOEH3ZYDMSYARJ5,220.0,210.0,"[""http://img5a.flixcart.com/image/pet-shampoo/...",False,Specifications of Sicons All Purpose Arnica Do...,No rating available,No rating available,Sicons,"{""product_specification""=>[{""key""=>""Pet Type"",..."


In [2]:
full_ds['image'][0]

'["http://img5a.flixcart.com/image/short/u/4/a/altht-3p-21-alisha-38-original-imaeh2d5vm5zbtgg.jpeg", "http://img5a.flixcart.com/image/short/p/j/z/altght4p-26-alisha-38-original-imaeh2d5kbufss6n.jpeg", "http://img5a.flixcart.com/image/short/p/j/z/altght4p-26-alisha-38-original-imaeh2d5npdybzyt.jpeg", "http://img5a.flixcart.com/image/short/z/j/7/altght-7-alisha-38-original-imaeh2d5jsz2ghd6.jpeg"]'

In [3]:
full_ds['description'][0]

"Key Features of Alisha Solid Women's Cycling Shorts Cotton Lycra Navy, Red, Navy,Specifications of Alisha Solid Women's Cycling Shorts Shorts Details Number of Contents in Sales Package Pack of 3 Fabric Cotton Lycra Type Cycling Shorts General Details Pattern Solid Ideal For Women's Fabric Care Gentle Machine Wash in Lukewarm Water, Do Not Bleach Additional Details Style Code ALTHT_3P_21 In the Box 3 shorts"

In [2]:
# check the values of each row for each column
full_ds.nunique(axis=0)

uniq_id                    20000
crawl_timestamp              371
product_url                20000
product_name               12676
product_category_tree       6466
pid                        19998
retail_price                2247
discounted_price            2448
image                      18589
is_FK_Advantage_product        2
description                17539
product_rating                36
overall_rating                36
brand                       3499
product_specifications     18825
dtype: int64

In [3]:
full_ds.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 20000 entries, 0 to 19999
Data columns (total 15 columns):
 #   Column                   Non-Null Count  Dtype  
---  ------                   --------------  -----  
 0   uniq_id                  20000 non-null  object 
 1   crawl_timestamp          20000 non-null  object 
 2   product_url              20000 non-null  object 
 3   product_name             20000 non-null  object 
 4   product_category_tree    20000 non-null  object 
 5   pid                      20000 non-null  object 
 6   retail_price             19922 non-null  float64
 7   discounted_price         19922 non-null  float64
 8   image                    19997 non-null  object 
 9   is_FK_Advantage_product  20000 non-null  bool   
 10  description              19998 non-null  object 
 11  product_rating           20000 non-null  object 
 12  overall_rating           20000 non-null  object 
 13  brand                    14136 non-null  object 
 14  product_specifications

In [4]:
df = full_ds[['uniq_id','product_name','description','brand','product_category_tree','image','product_specifications']]

In [5]:
df.head()

,uniq_id,product_name,description,brand,product_category_tree,image,product_specifications
0,c2d766ca982eca8304150849735ffef9,Alisha Solid Women's Cycling Shorts,Key Features of Alisha Solid Women's Cycling S...,Alisha,"[""Clothing >> Women's Clothing >> Lingerie, Sl...","[""http://img5a.flixcart.com/image/short/u/4/a/...","{""product_specification""=>[{""key""=>""Number of ..."
1,7f7036a6d550aaa89d34c77bd39a5e48,FabHomeDecor Fabric Double Sofa Bed,FabHomeDecor Fabric Double Sofa Bed (Finish Co...,FabHomeDecor,"[""Furniture >> Living Room Furniture >> Sofa B...","[""http://img6a.flixcart.com/image/sofa-bed/j/f...","{""product_specification""=>[{""key""=>""Installati..."
2,f449ec65dcbc041b6ae5e6a32717d01b,AW Bellies,Key Features of AW Bellies Sandals Wedges Heel...,AW,"[""Footwear >> Women's Footwear >> Ballerinas >...","[""http://img5a.flixcart.com/image/shoe/7/z/z/r...","{""product_specification""=>[{""key""=>""Ideal For""..."
3,0973b37acd0c664e3de26e97e5571454,Alisha Solid Women's Cycling Shorts,Key Features of Alisha Solid Women's Cycling S...,Alisha,"[""Clothing >> Women's Clothing >> Lingerie, Sl...","[""http://img5a.flixcart.com/image/short/6/2/h/...","{""product_specification""=>[{""key""=>""Number of ..."
4,bc940ea42ee6bef5ac7cea3fb5cfbee7,Sicons All Purpose Arnica Dog Shampoo,Specifications of Sicons All Purpose Arnica Do...,Sicons,"[""Pet Supplies >> Grooming >> Skin & Coat Care...","[""http://img5a.flixcart.com/image/pet-shampoo/...","{""product_specification""=>[{""key""=>""Pet Type"",..."


# Category Analysis

In [6]:
#Helper function to reformat the given text
def reformat(text: str) -> str:
  text = text.replace('[', '')
  text = text.replace(']', '')
  text = text.replace('"', '')
  return text

#df.loc[:, 'product_category_tree'] = df['product_category_tree'].apply(lambda x: reformat(x))
df['product_category_tree'] = df['product_category_tree'].apply(lambda x: reformat(x))

/tmp/ipykernel_3235/1039320457.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['product_category_tree'] = df['product_category_tree'].apply(lambda x: reformat(x))


In [7]:
# Finding the depth of the category trees
# Finding total number of categories in each level
cat_len = {}
for cat_tree in df.product_category_tree:
  number_of_categories = len(cat_tree.split(">>"))
  #print(number_of_categories)
  if number_of_categories not in cat_len:
    cat_len[number_of_categories] = 1
  else:
    cat_len[number_of_categories] += 1
print(cat_len)

{6: 3640, 4: 4765, 5: 4911, 1: 328, 3: 4419, 7: 778, 2: 1129, 8: 30}


**There are total 8 levels at max.**

In [8]:
temp_df = df['product_category_tree'].str.split('>>', expand=True)
temp_df.columns = ['c0_name', 'c1_name', 'c2_name', 'c3_name', 'c4_name', 'c5_name', 'c6_name', 'c7_name']
for col in temp_df.columns:
  temp_df[col] = temp_df[col].apply(lambda x: x.strip() if x else x)

**Considering only 4 levels from category tree**

In [9]:
#For our purposes we'll only consider the top 4 levels from category tree
temp_df =temp_df[['c0_name', 'c1_name', 'c2_name', 'c3_name']]
temp_df

,c0_name,c1_name,c2_name,c3_name
0,Clothing,Women's Clothing,"Lingerie, Sleep & Swimwear",Shorts
1,Furniture,Living Room Furniture,Sofa Beds & Futons,FabHomeDecor Fabric Double Sofa Bed (Finish Co...
2,Footwear,Women's Footwear,Ballerinas,AW Bellies
3,Clothing,Women's Clothing,"Lingerie, Sleep & Swimwear",Shorts
4,Pet Supplies,Grooming,Skin & Coat Care,Shampoo
...,...,...,...,...
19995,Baby Care,Baby & Kids Gifts,Stickers,WallDesign Stickers
19996,Baby Care,Baby & Kids Gifts,Stickers,Wallmantra Stickers
19997,Baby Care,Baby & Kids Gifts,Stickers,Elite Collection Stickers
19998,Baby Care,Baby & Kids Gifts,Stickers,Elite Collection Stickers


In [10]:
# concatenating df1 and df2 along rows
df_with_cat = pd.concat([df, temp_df], axis=1)
df_with_cat = df_with_cat.drop('product_category_tree', axis=1)

In [11]:
df_with_cat.head()

,uniq_id,product_name,description,brand,image,product_specifications,c0_name,c1_name,c2_name,c3_name
0,c2d766ca982eca8304150849735ffef9,Alisha Solid Women's Cycling Shorts,Key Features of Alisha Solid Women's Cycling S...,Alisha,"[""http://img5a.flixcart.com/image/short/u/4/a/...","{""product_specification""=>[{""key""=>""Number of ...",Clothing,Women's Clothing,"Lingerie, Sleep & Swimwear",Shorts
1,7f7036a6d550aaa89d34c77bd39a5e48,FabHomeDecor Fabric Double Sofa Bed,FabHomeDecor Fabric Double Sofa Bed (Finish Co...,FabHomeDecor,"[""http://img6a.flixcart.com/image/sofa-bed/j/f...","{""product_specification""=>[{""key""=>""Installati...",Furniture,Living Room Furniture,Sofa Beds & Futons,FabHomeDecor Fabric Double Sofa Bed (Finish Co...
2,f449ec65dcbc041b6ae5e6a32717d01b,AW Bellies,Key Features of AW Bellies Sandals Wedges Heel...,AW,"[""http://img5a.flixcart.com/image/shoe/7/z/z/r...","{""product_specification""=>[{""key""=>""Ideal For""...",Footwear,Women's Footwear,Ballerinas,AW Bellies
3,0973b37acd0c664e3de26e97e5571454,Alisha Solid Women's Cycling Shorts,Key Features of Alisha Solid Women's Cycling S...,Alisha,"[""http://img5a.flixcart.com/image/short/6/2/h/...","{""product_specification""=>[{""key""=>""Number of ...",Clothing,Women's Clothing,"Lingerie, Sleep & Swimwear",Shorts
4,bc940ea42ee6bef5ac7cea3fb5cfbee7,Sicons All Purpose Arnica Dog Shampoo,Specifications of Sicons All Purpose Arnica Do...,Sicons,"[""http://img5a.flixcart.com/image/pet-shampoo/...","{""product_specification""=>[{""key""=>""Pet Type"",...",Pet Supplies,Grooming,Skin & Coat Care,Shampoo


In [12]:
df_with_cat['c0_name'].value_counts()

c0_name
Clothing                                               6198
Jewellery                                              3531
Footwear                                               1227
Mobiles & Accessories                                  1099
Automotive                                             1012
                                                       ... 
Mast & Harbour Gold Synthetic  Clutch                     1
Asics Gel-Cumulus 17 Running Shoes                        1
Glacier Running Shoes                                     1
Starsy Solid Women's Round Neck Green T-Shirt             1
Areon Luxurious Fragrance Long Lasting Car,Home,...       1
Name: count, Length: 265, dtype: int64

In [13]:
df_with_cat['c1_name'].value_counts()

c1_name
Women's Clothing                             3901
Men's Clothing                               1773
Necklaces & Chains                           1606
Accessories & Spare parts                     925
Tablet Accessories                            801
                                             ... 
CEAT 2.75-18 Gripp TT Tube Tyre                 1
CEAT 2.75-17 Secura F85 TL Tube Less Tyre       1
CEAT 140/70-17 Zoom XL TL Tube Less Tyre        1
CEAT 3.50-10 Milaze TT Tube Tyre                1
Home Utilities                                  1
Name: count, Length: 216, dtype: int64

In [14]:
df_with_cat['c2_name'].value_counts()

c2_name
Western Wear                                           1981
Necklaces                                              1588
Lingerie, Sleep & Swimwear                             1208
T-Shirts                                                903
Cases & Covers                                          796
                                                       ... 
RosesNRibbons Rhodium Garnet Sterling Silver              1
RosesNRibbons Rhodium Garnet, Amethyst, Citrine,...       1
Sanaa Creations Alloy                                     1
Gas Cylinder Trolleys                                     1
Dyna Mobile Accessories                                   1
Name: count, Length: 874, dtype: int64

In [15]:
df_with_cat['c3_name'].value_counts()

c3_name
Shirts, Tops & Tunics                                  1249
Bras                                                   1036
Car Interior                                            659
Dresses & Skirts                                        620
Combos                                                  375
                                                       ... 
Rajdoot Casual Shoes                                      1
Adishma Printed Cushions Cover (40 cm*40 cm, Blue)        1
Ally The Creations Printed Cushions Cover (Pack ...       1
Style Homez Polka Cushions Cover (45 cm*45 cm, M...       1
Fun To See Stickers                                       1
Name: count, Length: 2347, dtype: int64

# Extracting Product Attributes

In [16]:
#Extracting attributes from product specifications
import json
from typing import List, Dict

import jsonpickle
import pandas as pd
import re

import numpy as np
SPEC_MATCH_ONE = re.compile("(.*?)\\[(.*)\\](.*)")
SPEC_MATCH_TWO = re.compile("(.*?)=>\"(.*?)\"(.*?)=>\"(.*?)\"(.*)")

def parse_spec(specification: str):
    if pd.isna(specification):
      return None
    m = SPEC_MATCH_ONE.match(specification)
    out = {}
    position = 0
    if m is not None and m.group(2) is not None:
        phrase = ''
        for c in m.group(2):
            if c == '}':
                m2 = SPEC_MATCH_TWO.match(phrase)
                if m2 and m2.group(2) is not None and m2.group(4) is not None:
                    out[m2.group(2)]=m2.group(4)
                phrase = ''
            else:
                phrase += c
    json_string = jsonpickle.encode(out)
    return json_string

In [19]:
df_with_cat['attributes'] = df_with_cat['product_specifications'].apply(parse_spec)
df_with_cat['attributes']

0        {"Number of Contents in Sales Package": "Pack ...
1        {"Installation & Demo Details": "Installation ...
2        {"Ideal For": "Women", "Occasion": "Casual", "...
3        {"Number of Contents in Sales Package": "Pack ...
4        {"Pet Type": "Dog", "Brand": "Sicons", "Quanti...
                               ...                        
19995    {"Number of Contents in Sales Package": "Pack ...
19996    {"Number of Contents in Sales Package": "Pack ...
19997    {"Number of Contents in Sales Package": "Pack ...
19998    {"Number of Contents in Sales Package": "Pack ...
19999    {"Number of Contents in Sales Package": "Pack ...
Name: attributes, Length: 20000, dtype: object

# Downloading Images from product image url into GCS

In [ ]:
from google.cloud import storage
from google.cloud.storage import Bucket

IMAGE_BUCKET = 'genai-product-catalog' # UPDATE
GCS_IMAGE_FOLDER = 'flipkart_Nov14' # UPDATE

def create_gcs_bucket(bucket_name: str):
  storage_client = storage.Client()
  exists = Bucket(storage_client, IMAGE_BUCKET).exists()
  if exists:
    print("Bucket exists")
  else:
    print("Bucket does not exist")
    # Creates the new bucket
    bucket = storage_client.create_bucket(IMAGE_BUCKET)
    print(f"Bucket {bucket.name} created.")

In [ ]:
create_gcs_bucket(IMAGE_BUCKET)

Bucket exists


In [ ]:
import urllib.request, urllib.error
#from numpy import NaN

#Formating Imagelist string into list of image urls
def extract_url(image_list: str) -> List[str]:
  image_list = image_list.replace('[', '')
  image_list = image_list.replace(']', '')
  image_list = image_list.replace('"', '')
  #image_list = image_list.apply(lambda x: reformat(x))
  image_urls = image_list.split(',')
  return image_urls

#Dowlonding image from flipkart url into gcs bucket
def download_image(image_url, image_file_name, destination_blob_name):
  storage_client = storage.Client()
  image_found_flag = False
  try:
    urllib.request.urlretrieve(image_url, image_file_name)
    bucket = storage_client.bucket(IMAGE_BUCKET)
    blob = bucket.blob(destination_blob_name)
    blob.upload_from_filename(image_file_name)
    print(
        f"File {image_file_name} uploaded to {destination_blob_name}."
    )
    image_found_flag = True
  except urllib.error.URLError:
    print("URLError exception")
  except urllib.error.HTTPError:
    print("HTTPError exception")
  except urllib.error.HTTPException:
    print("HTTPException exception")
  except:
    print("Unknown exception")
  return image_found_flag

In [ ]:
#Get Image URI
def get_product_image(df):
  products_with_no_image_count = 0
  products_with_no_image = []
  gcs_image_url = []
  image_found_flag = False

  for id, image_list in zip(df['uniq_id'], df['image']):

    if pd.isnull(image_list): #No image url
      print("WARNING: No image url: product ", id)
      products_with_no_image_count += 1
      products_with_no_image.append(id)
      gcs_image_url.append(None)
      continue

    image_urls = extract_url(image_list)
    for index in range(len(image_urls)):
      image_url = image_urls[index]
      image_file_name = '{}_{}.jpg'.format(id, index)
      destination_blob_name = GCS_IMAGE_FOLDER+'/'+image_file_name
      image_found_flag = download_image(image_url, image_file_name, destination_blob_name)
      if image_found_flag:
        gcs_image_url.append('gs://'+IMAGE_BUCKET+'/'+destination_blob_name)
        break
    if not image_found_flag:
      print("WARNING: No image: product ", id)
      products_with_no_image_count += 1
      products_with_no_image.append(id)
      gcs_image_url.append(None)

  #appending gcs image uri into dataframe
  gcs_image_loc = pd.DataFrame(gcs_image_url)
  gcs_image_loc.columns = ["image_uri"]

  df_with_gcs_image_uri = pd.concat([df, gcs_image_loc], axis=0)
  df_with_gcs_image_uri = df_with_gcs_image_uri.drop('image', axis=1)

  return df_with_gcs_image_uri

In [ ]:
df_with_gcs_image_uri = get_product_image(df_with_cat)

In [ ]:
df_with_gcs_image_uri.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 20000 entries, 0 to 19999
Data columns (total 12 columns):
 #   Column                  Non-Null Count  Dtype 
---  ------                  --------------  ----- 
 0   uniq_id                 20000 non-null  object
 1   product_name            20000 non-null  object
 2   description             20000 non-null  object
 3   brand                   14136 non-null  object
 4   image                   19997 non-null  object
 5   product_specifications  19986 non-null  object
 6   c0_name                 20000 non-null  object
 7   c1_name                 19672 non-null  object
 8   c2_name                 18543 non-null  object
 9   c3_name                 14124 non-null  object
 10  attributes              19986 non-null  object
 11  image_uri               18360 non-null  object
dtypes: object(12)
memory usage: 2.0+ MB


In [ ]:
#Filtering null values
non_null_image_df = df_with_gcs_image_uri[df_with_gcs_image_uri['image_uri'].notnull()]
non_null_image_df = non_null_image_df[non_null_image_df['description'].notnull()]

In [ ]:
non_null_image_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 18360 entries, 0 to 19999
Data columns (total 12 columns):
 #   Column                  Non-Null Count  Dtype 
---  ------                  --------------  ----- 
 0   uniq_id                 18360 non-null  object
 1   product_name            18360 non-null  object
 2   description             18360 non-null  object
 3   brand                   13476 non-null  object
 4   image                   18360 non-null  object
 5   product_specifications  18347 non-null  object
 6   c0_name                 18360 non-null  object
 7   c1_name                 18064 non-null  object
 8   c2_name                 16940 non-null  object
 9   c3_name                 12829 non-null  object
 10  attributes              18347 non-null  object
 11  image_uri               18360 non-null  object
dtypes: object(12)
memory usage: 1.8+ MB


In [ ]:
#Dropping redundant columns
non_null_image_df = non_null_image_df.drop(['image','product_specifications'], axis=1)
non_null_image_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 18360 entries, 0 to 19999
Data columns (total 10 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   uniq_id       18360 non-null  object
 1   product_name  18360 non-null  object
 2   description   18360 non-null  object
 3   brand         13476 non-null  object
 4   c0_name       18360 non-null  object
 5   c1_name       18064 non-null  object
 6   c2_name       16940 non-null  object
 7   c3_name       12829 non-null  object
 8   attributes    18347 non-null  object
 9   image_uri     18360 non-null  object
dtypes: object(10)
memory usage: 1.5+ MB


In [ ]:
non_null_image_df.to_csv('flipkart_preprocessed.csv', header=False, index=False)

In [ ]:
non_null_image_df.head()

,uniq_id,product_name,description,brand,c0_name,c1_name,c2_name,c3_name,attributes,image_uri
0,c2d766ca982eca8304150849735ffef9,Alisha Solid Women's Cycling Shorts,key feature alisha solid woman cycling short c...,Alisha,Clothing,Women's Clothing,"Lingerie, Sleep & Swimwear",Shorts,"{""Number of Contents in Sales Package"": ""Pack ...",gs://genai-product-catalog/flipkart_20k_oct26/...
1,7f7036a6d550aaa89d34c77bd39a5e48,FabHomeDecor Fabric Double Sofa Bed,fabhomedecor fabric double sofa bed finish col...,FabHomeDecor,Furniture,Living Room Furniture,Sofa Beds & Futons,FabHomeDecor Fabric Double Sofa Bed (Finish Co...,"{""Installation & Demo Details"": ""Installation ...",gs://genai-product-catalog/flipkart_20k_oct26/...
2,f449ec65dcbc041b6ae5e6a32717d01b,AW Bellies,key feature aw belly sandal wedge heel casual ...,AW,Footwear,Women's Footwear,Ballerinas,AW Bellies,"{""Ideal For"": ""Women"", ""Occasion"": ""Casual"", ""...",gs://genai-product-catalog/flipkart_20k_oct26/...
3,0973b37acd0c664e3de26e97e5571454,Alisha Solid Women's Cycling Shorts,key feature alisha solid woman cycling short c...,Alisha,Clothing,Women's Clothing,"Lingerie, Sleep & Swimwear",Shorts,"{""Number of Contents in Sales Package"": ""Pack ...",gs://genai-product-catalog/flipkart_20k_oct26/...
5,c2a17313954882c1dba461863e98adf2,Eternal Gandhi Super Series Crystal Paper Weig...,key feature eternal gandhi super series crysta...,Eternal Gandhi,Eternal Gandhi Super Series Crystal Paper Weig...,None,None,None,"{""Model Name"": ""Gandhi Paper Weight Mark V"", ""...",gs://genai-product-catalog/flipkart_20k_oct26/...


In [ ]:
#renaming column name to match embedding generation code
non_null_image_df.rename(columns={'uniq_id':'id'}, inplace=True)

# Upload preprocessed data into BigQuery

In [ ]:
from google.cloud import bigquery

def create_table(client, table_id, schema):
    table = bigquery.Table(table_id, schema=schema)
    table = client.create_table(table,exists_ok=True)  # Make an API request
    print(
        "Created table {}.{}.{}".format(table.project, table.dataset_id, table.table_id)
    )
def upload_df_into_bq(client, table_id, df):
    #df.to_gbq(table_id, PROJECT, if_exists='replace', progress_bar=True)
    job_config = bigquery.LoadJobConfig(schema=schema)
    job_config.write_disposition = bigquery.WriteDisposition.WRITE_TRUNCATE
    #job_config.skip_leading_rows = 1
    job_config.autodetect = False
    job_config.source_format = 'CSV'
    job = client.load_table_from_dataframe(df, table_id, job_config=job_config)
    job.result()
    print(
        "Uploaded dataframe into table {}.{}".format(PROJECT, table_id)
    )

In [ ]:
PROJECT = 'solutions-2023-mar-107' # UPDATE
LOCATION = 'us-central1' # UPDATE
table_id = 'solutions-2023-mar-107.flipkart.preprocessed_data' # UPDATE

schema = [
    bigquery.SchemaField('id', 'STRING', mode='REQUIRED'),
    bigquery.SchemaField('product_name', 'STRING', mode='REQUIRED'),
    bigquery.SchemaField('description', 'STRING', mode='REQUIRED'),
    bigquery.SchemaField('brand', 'STRING', mode='NULLABLE'),
    bigquery.SchemaField('c0_name', 'STRING', mode='NULLABLE'),
    bigquery.SchemaField('c1_name', 'STRING', mode='NULLABLE'),
    bigquery.SchemaField('c2_name', 'STRING', mode='NULLABLE'),
    bigquery.SchemaField('c3_name', 'STRING', mode='NULLABLE'),
    bigquery.SchemaField('attributes', 'JSON', mode='NULLABLE'),
    bigquery.SchemaField('image_uri', 'STRING', mode='REQUIRED')
]
client = bigquery.Client(PROJECT)

create_table(client, table_id, schema)

upload_df_into_bq(client, table_id, non_null_image_df)


**Adding new (empty) columns for text & image embeddings**

In [ ]:
table = client.get_table(table_id)
original_schema = table.schema
new_schema = original_schema[:]  # Creates a copy of the schema.
new_schema.append(bigquery.SchemaField('text_embedding', 'FLOAT', mode='REPEATED'))
new_schema.append(bigquery.SchemaField('image_embedding', 'FLOAT', mode='REPEATED'))

table.schema = new_schema
table = client.update_table(table, ["schema"])  # Make an API request.

if len(table.schema) == len(original_schema) + 2 == len(new_schema):
    print("Two new columns have been added.")
else:
    print("Something went wrong.")

Two new columns have been added.
